In [1]:
from keras.datasets import boston_housing

In [2]:
(trainData, trainTargets), (testData, testTargets) = boston_housing.load_data()

57344/57026 [==============================] - 0s 7us/step


In [3]:
trainData.shape, testData.shape

((404, 13), (102, 13))

In [11]:
mean = trainData.mean(axis=0)
trainData -= mean
std = trainData.std(axis=0)
trainData /= std

testData -= mean
testData /= std

In [8]:
from keras import models
from keras import layers

def build_model():
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu'))